# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [24]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo

In [26]:

model_info_gpt_oss = ModelInfo(vision=False, function_calling=True, json_output=True, structured_output=True, multiple_system_messages=False, family="gpt-oss")
# Initialize the model client
model_client = OllamaChatCompletionClient(
    model="gpt-oss:20b",
    model_info=model_info_gpt_oss,
    parallel_tool_calls=False,
)

In [33]:
# Create specialized agents
researcher   = AssistantAgent(name="Researcher", system_message="Gather and summarize factual info.", model_client=model_client)
factchecker  = AssistantAgent(name="FactChecker", system_message="Verify facts and cite sources.", model_client=model_client)
critic       = AssistantAgent(name="Critic", system_message="Critique clarity and logic.", model_client=model_client)
summarizer   = AssistantAgent(name="Summarizer", system_message="Condense into a brief executive summary.", model_client=model_client)
editor       = AssistantAgent(name="Editor", system_message="Do some language polishing and signal APPROVED when done.", model_client=model_client)

# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=10)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=[researcher, factchecker, critic, summarizer, editor],
    termination_condition=termination
)

In [34]:
# Create the host agent with deep dive capabilities
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")

host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

In [35]:
async def run_deepdive(topic: str):
    result = await host.run(task=f"Deep dive on: {topic}")
    print("🔍 DeepDive result:\n", result)
    return result  # Return the result for further exploration

# Set your research topic
topic = "Impacts of Model Context Protocol on Agentic AI"

# Run the deep dive
result = await run_deepdive(topic)

🔍 DeepDive result:
 messages=[TextMessage(id='175bc84e-26cf-4313-8f58-77eab3e60999', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 6, 16, 39, 19, 707813, tzinfo=datetime.timezone.utc), content='Deep dive on: Impacts of Model Context Protocol on Agentic AI', type='TextMessage'), ToolCallRequestEvent(id='de453797-834d-4bca-bda9-20eb400bcba9', source='Host', models_usage=RequestUsage(prompt_tokens=151, completion_tokens=166), metadata={}, created_at=datetime.datetime(2025, 9, 6, 16, 39, 38, 695083, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='1', arguments='{"task": "Impacts of Model Context Protocol on Agentic AI"}', name='DeepDive')], type='ToolCallRequestEvent'), TextMessage(id='1d4f5868-ba43-4d1c-bc9b-2140ff94f034', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 6, 16, 39, 38, 695530, tzinfo=datetime.timezone.utc), content='Impacts of Model Context Protocol on Agentic AI', type='TextMessage'), Tex

## Explore the Results

Now that we have the results, let's explore them. The cells below will help you analyze the output.

In [36]:
# Display the full result structure
print("Result type:", type(result))
print("\nResult attributes:")
print([attr for attr in dir(result) if not attr.startswith('_')])

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']


In [37]:
# Clean up resources
await model_client.close()

In [38]:
print(len(result.messages))
print(result.stop_reason)
print(result.messages[-1].content)  # Print the content of the last message

10
None
Researcher: **Impacts of the Model Context Protocol (MCP) on Agentic AI**

| Domain | How MCP Alters Agentic AI | Practical Implications | Key Caveats |
|--------|---------------------------|------------------------|-------------|
| **Context Fidelity** | MCP enforces a strict, structured flow of contextual data between the agent’s internal state and the external environment. | • Agents can maintain coherent long‑term plans.<br>• Reduced risk of “hallucinating” facts that are irrelevant to the current context. | • Requires that all external inputs be tagged and verified; otherwise the protocol can block useful data. |
| **Goal Alignment & Safety** | By keeping a clear, verifiable log of the agent’s goals and the context in which it operates, MCP allows alignment checks to be performed automatically. | • On‑line verification that an agent’s actions are consistent with its declared goals.<br>• Easier auditability of decision‑making processes. | • Alignment logic itself must be ro

In [ ]:
# Lightweight internet research tool (uses requests + BeautifulSoup by default)
import requests
from bs4 import BeautifulSoup
from typing import List, Dict

class InternetResearchTool:
    def __init__(self, user_agent: str = 'research-agent/1.0') -> None:
        self.headers = {'User-Agent': user_agent}

    def search(self, query: str, max_results: int = 5) -> List[Dict]:
        """Perform a DuckDuckGo HTML search and return a list of results with title, link, and snippet.
        This is a lightweight fallback when a commercial search API isn't configured.
        """
        params = {'q': query}
        r = requests.get('https://html.duckduckgo.com/html/', params=params, headers=self.headers, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        results = []
        for a in soup.select('a.result__a')[:max_results]:
            title = a.get_text()
            link = a.get('href')
            snippet_el = a.find_parent('div', class_='result').select_one('.result__snippet')
            snippet = snippet_el.get_text() if snippet_el else ''
            results.append({'title': title, 'link': link, 'snippet': snippet})
        return results

    def fetch_page(self, url: str) -> str:
        r = requests.get(url, headers=self.headers, timeout=15)
        r.raise_for_status()
        return r.text

In [ ]:
# Instantiate internet research tool and attach to FactChecker
internet_tool = InternetResearchTool()
# Attach as a tool to the FactChecker agent so it can call it when needed
factchecker.tools = getattr(factchecker, 'tools', []) + [internet_tool]

# Example: run the tool (blocking) in a notebook-friendly way
from asyncio import to_thread
async def demo_search(q):
    results = await to_thread(internet_tool.search, q, 3)
    for i, r in enumerate(results, 1):
        print(i, r['title'])
        print('  ', r['link'])
        print('  ', r['snippet'])

# Try a quick search
await demo_search('Model Context Protocol agentic AI')